In [ ]:
import os
from pathlib import Path
import pandas as pd
import openpyxl
from datetime import datetime, timedelta

In [ ]:
R_Exceles = r"\\Servernas\AYC2\(Z)RSERVER(Z)\01_Ing. YESID\FRANK\GRUPO1"
R_PDFs = r"\\Servernas\Server2\12_Servicios de Salud\2025\CTC\ACTAS  COMITES"
R_Salida = r"\\Servernas\Server2\12_Servicios de Salud\2025\CTC"

In [ ]:
import glob
import pandas as pd
from datetime import datetime
import re
import numpy as np

# Get all XLS files from the directory
excel_files = glob.glob(os.path.join(R_Exceles, "*.XLS"))

print(f"Total de archivos encontrados: {len(excel_files)}")

data_list = []
error_count = 0

for file_path in excel_files:
    try:
        # Leer con xlrd para obtener datos
        df_full = pd.read_excel(file_path, sheet_name=0, header=None, engine='xlrd')
        
        # Buscar la fecha en todo el archivo (puede estar en cualquier lugar)
        fecha_reporte = None
        for idx, row in df_full.iterrows():
            for col_idx, cell_value in enumerate(row):
                if isinstance(cell_value, (pd.Timestamp, datetime)):
                    fecha_reporte = cell_value
                    break
            if fecha_reporte:
                break
        
        # Si no encontró como datetime, buscar fechas en formato texto
        if not fecha_reporte:
            patron_fecha = r'\d{1,2}/\d{1,2}/\d{4}|\d{4}-\d{1,2}-\d{1,2}'
            for idx, row in df_full.iterrows():
                for cell_value in row:
                    if isinstance(cell_value, str) and re.search(patron_fecha, cell_value):
                        try:
                            fecha_reporte = pd.to_datetime(cell_value)
                            break
                        except:
                            pass
                if fecha_reporte:
                    break
        
        # Leer los datos (columnas A a I, header en fila 3)
        df_temp = pd.read_excel(file_path, sheet_name=0, header=2, usecols='A:I', engine='xlrd')
        
        # PASO 1: Eliminar columnas completamente vacías (Unnamed)
        df_temp = df_temp.dropna(axis=1, how='all')
        
        # PASO 2: Eliminar filas completamente vacías
        df_temp = df_temp.dropna(axis=0, how='all')
        
        # PASO 3: Eliminar filas donde TODAS las columnas de datos están vacías
        df_temp = df_temp.loc[df_temp.notna().any(axis=1)]
        
        # PASO 4: Limpiar columnas específicas que suelen tener problemas
        if 'CANT.' in df_temp.columns:
            df_temp['CANT.'] = pd.to_numeric(df_temp['CANT.'], errors='coerce')
            df_temp['CANT.'] = df_temp['CANT.'].fillna(method='ffill')
        
        if 'ESTADO' in df_temp.columns:
            df_temp['ESTADO'] = df_temp['ESTADO'].str.upper().str.strip()
            df_temp.loc[~df_temp['ESTADO'].isin(['AUTORIZADO', 'NEGADO']), 'ESTADO'] = np.nan
            df_temp['ESTADO'] = df_temp['ESTADO'].fillna(method='ffill')
        
        # PASO 5: Eliminar filas duplicadas completamente
        df_temp = df_temp.drop_duplicates(subset=None, keep='first')
        
        # PASO 6: Eliminar filas donde ESTADO sea NaN después de limpieza
        if 'ESTADO' in df_temp.columns:
            df_temp = df_temp[df_temp['ESTADO'].notna()]
        
        # PASO 7: Llenar ACTA si está vacía (forward fill desde arriba)
        if 'ACTA' in df_temp.columns:
            df_temp['ACTA'] = df_temp['ACTA'].fillna(method='ffill')
        
        # PASO 8: Eliminar filas donde ACTA sea NaN o vacía después de llenar
        if 'ACTA' in df_temp.columns:
            df_temp = df_temp[df_temp['ACTA'].notna()]
            df_temp = df_temp[df_temp['ACTA'].astype(str).str.strip() != '']
        
        # PASO 9: Eliminar filas donde CASO sea NaN o vacía
        if 'CASO' in df_temp.columns:
            df_temp = df_temp[df_temp['CASO'].notna()]
            df_temp = df_temp[df_temp['CASO'].astype(str).str.strip() != '']
        
        # PASO 10: Agregar columnas de fecha y archivo
        df_temp['Fecha-Reporte'] = fecha_reporte
        df_temp['Nombre-Archivo'] = os.path.basename(file_path)
        
        # Solo agregar si tiene registros válidos
        if len(df_temp) > 0:
            data_list.append(df_temp)
            print(f"✓ Procesado: {os.path.basename(file_path)} | Fecha: {fecha_reporte} | Filas: {len(df_temp)}")
        else:
            print(f"⚠️ Sin datos válidos: {os.path.basename(file_path)}")
    
    except Exception as e:
        error_count += 1
        print(f"✗ Error en {os.path.basename(file_path)}: {str(e)[:100]}")

print(f"\n--- Resumen ---")
print(f"Archivos procesados: {len(data_list)}")
print(f"Archivos con error: {error_count}")

if len(data_list) > 0:
    # Concatenar de forma inteligente
    df_unificado = pd.concat(data_list, axis=0, ignore_index=True, sort=False)
    
    # Limpiar columnas Unnamed vacías
    unnamed_cols = [col for col in df_unificado.columns if 'Unnamed' in str(col)]
    for col in unnamed_cols:
        if df_unificado[col].isna().sum() == len(df_unificado):
            df_unificado = df_unificado.drop(columns=[col])
    
    # Eliminar filas completamente vacías
    df_unificado = df_unificado.dropna(axis=0, how='all')
    
    # Validación final: eliminar filas sin ESTADO válido
    if 'ESTADO' in df_unificado.columns:
        df_unificado = df_unificado[df_unificado['ESTADO'].isin(['AUTORIZADO', 'NEGADO'])]
    
    # Validación final: eliminar filas sin ACTA válida
    if 'ACTA' in df_unificado.columns:
        df_unificado = df_unificado[df_unificado['ACTA'].notna()]
        df_unificado = df_unificado[df_unificado['ACTA'].astype(str).str.strip() != '']
    
    # Validación final: eliminar filas sin CASO válido
    if 'CASO' in df_unificado.columns:
        df_unificado = df_unificado[df_unificado['CASO'].notna()]
        df_unificado = df_unificado[df_unificado['CASO'].astype(str).str.strip() != '']
    
    # Reorganizar columnas (Fecha-Reporte y Nombre-Archivo al final)
    cols = [col for col in df_unificado.columns if col not in ['Fecha-Reporte', 'Nombre-Archivo']]
    cols.extend(['Fecha-Reporte', 'Nombre-Archivo'])
    df_unificado = df_unificado[cols]
    
    # Guardar
    output_path = r"\\Servernas\Server2\12_Servicios de Salud\2025\CTC\Reporte_unificado_CTC.xlsx"
    df_unificado.to_excel(output_path, index=False, engine='openpyxl')
    
    print(f"\n✓ Reporte unificado guardado en: {output_path}")
    print(f"  Total de filas: {len(df_unificado)}")
    print(f"  Total de columnas: {len(df_unificado.columns)}")
    print(f"  Columnas: {list(df_unificado.columns)}")
    print(f"\nValidación de ESTADO:")
    print(df_unificado['ESTADO'].value_counts())
    print(f"\nACTAs únicas: {df_unificado['ACTA'].nunique()}")
    print(f"CASOs únicos: {df_unificado['CASO'].nunique()}")
else:
    print("⚠️ No se procesó ningún archivo")

In [ ]:
import shutil

# Filtrar registros con estado "NEGADO"
df_negados = df_unificado[df_unificado['ESTADO'].str.upper().str.contains('NEGADO', na=False)].copy()

print(f"Total de registros NEGADOS: {len(df_negados)}")

# Obtener ACTAs únicas de los registros negados
actas_negadas = df_negados['ACTA'].dropna().unique()
print(f"ACTAs únicas con estado NEGADO: {len(actas_negadas)}")

# Preparar rutas
carpeta_origen_pdfs = R_PDFs
carpeta_destino_pdfs = r"\\Servernas\AYC2\(Z)RSERVER(Z)\01_Ing. YESID\FRANK\PDF Negados"

# Crear carpeta de destino si no existe
os.makedirs(carpeta_destino_pdfs, exist_ok=True)

# Buscar y copiar PDFs

pdfs_copiados = []
pdfs_no_encontrados = []

for acta in actas_negadas:
    # Convertir ACTA a entero para formato sin decimales
    try:
        acta_num = int(acta)
        nombre_pdf = f"ACTAS COMITE {acta_num}.pdf"
        ruta_origen = os.path.join(carpeta_origen_pdfs, nombre_pdf)
        ruta_destino = os.path.join(carpeta_destino_pdfs, nombre_pdf)
        
        # Verificar si el PDF existe y copiarlo
        if os.path.exists(ruta_origen):
            shutil.copy2(ruta_origen, ruta_destino)
            pdfs_copiados.append(nombre_pdf)
            print(f"✓ Copiado: {nombre_pdf}")
        else:
            pdfs_no_encontrados.append(nombre_pdf)
            print(f"✗ No encontrado: {nombre_pdf}")
    except:
        print(f"⚠️ Error procesando ACTA: {acta}")

print(f"\n--- Resumen de Copia de PDFs ---")
print(f"PDFs copiados exitosamente: {len(pdfs_copiados)}")
print(f"PDFs no encontrados: {len(pdfs_no_encontrados)}")
print(f"Destino: {carpeta_destino_pdfs}")

# Mostrar algunos casos negados para verificación
if len(df_negados) > 0:
    print(f"\nPrimeros registros NEGADOS:")
    print(df_negados[['ACTA', 'CASO', 'NOMBRES Y APELLIDOS', 'ESTADO', 'Nombre-Archivo']].head(10))